In [ ]:
using RCall
using DataFrames
using Plots

# Set CRAN mirror and install package
R"""
options(repos = c(CRAN = "https://cloud.r-project.org"))
if (!require('palmerpenguins')) install.packages('palmerpenguins')
library(palmerpenguins)
"""

# Get the penguins dataset into Julia
penguins = rcopy(R"penguins")

# so data, so adorable
 println(describe(penguins))

# just species
s = penguins[:, :species]

cols = penguins[:, [:species, :island, :sex]]

describe(cols)

reviews_locations = select(penguins, Cols(startswith("bill")))

describe(reviews_locations)

scatter(penguins[:, :bill_length_mm], penguins[:, :bill_depth_mm], group=penguins[:, :species])



In [26]:
#move bill_length_mm to after sex
select!(penguins, :species, :island, :sex, :bill_length_mm, :bill_depth_mm, :body_mass_g)

describe(penguins)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,species,,Adelie,,Gentoo,0,"CategoricalValue{String, UInt32}"
2,island,,Biscoe,,Torgersen,0,"CategoricalValue{String, UInt32}"
3,sex,,female,,male,11,"Union{Missing, CategoricalValue{String, UInt32}}"
4,bill_length_mm,43.9219,32.1,44.45,59.6,2,"Union{Missing, Float64}"
5,bill_depth_mm,17.1512,13.1,17.3,21.5,2,"Union{Missing, Float64}"
6,body_mass_g,4201.75,2700,4050.0,6300,2,"Union{Missing, Int64}"
